Последовательность выполнения:

1. Выберите на каком-нибудь сайте объявлений или в интернет-магазине интересующую вас рубрику самого нижнего уровня (например, Для дома и дачи – Мебель и интерьер –
Кровати, диваны и кресла – Диваны).
2. Убедитесь в том, что сайт учел в выдаче ваше местоположение (если нет – укажите его).
3. Обратите внимание на адрес полученной вами страницы и на то, как он изменяется при переходе к следующей странице выдачи (где и как указывается номер страницы), зафиксируйте это.
4. Откройте код страницы выдачи (F12 или правой кнопкой мыши).
Выберите ключевые элементы блока записи об одном объявлении так, чтобы вы могли выделить этот блок из всего текста: например, это может быть определенный тег, с определенным набором атрибутов.


## Взял сайт Крым Мебель

In [ ]:
from bs4 import BeautifulSoup
import requests

url = 'https://almaz-meb.ru/divany/divan-kembridzh/'  # конкретный товар
headers = {'User-Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Название
title = soup.find('h1', class_='ut2-pb__title')
print("Название:", title.get_text(strip=True) if title else "не найдено")

# Цена и ID
price_tag = soup.find('span', id=lambda x: x and x.startswith('sec_discounted_price_'))
if price_tag:
    print("Цена:", price_tag.get_text(strip=True))
    print("ID:", price_tag.get('id').split('_')[-1])
else:
    print("Цена не найдена")

# Код товара
code_tag = soup.find('span', id=lambda x: x and x.startswith('product_code_'))
print("Код товара:", code_tag.get_text(strip=True) if code_tag else "не найден")


In [ ]:
from bs4 import BeautifulSoup as BeSo
import requests
import re

url = 'https://almaz-meb.ru/divany/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15'
}

# Проверяем отвечает ли сайт
response = requests.get(url, headers=headers)
print(f"Статус: {response.status_code}")
soup = BeSo(response.text, "html.parser")

# Ссылки на товары
links = [a["href"] for a in soup.find_all("a", class_="product-title", href=True)]

#Сортировка по цене
min_price = 3000
max_price = 50000

# Будем парсить 4 товара
for link in links[:4]:
    if link.startswith("/"):
        product_url = "https://almaz-meb.ru" + link
    else:
        product_url = link
     #Отправка запроса к старанице товара   
    res = requests.get(product_url, headers=headers)
    p_soup = BeSo(res.text, "html.parser")

    # Цена и ID
    price_tag = p_soup.find("span", id=lambda x: x and x.startswith('sec_discounted_price_'))
    if price_tag:
        price_text = price_tag.get_text(strip= True) #Уберём пробелы
        price = int(re.sub(r'\D', '', price_text)) #Преобразуем числа в строку
        element_id = price_tag.get("id")
        item_id = element_id.split("_")[-1]
        print(f"ID: {item_id}")
        print(f"Цена: {price}")
    else:
        print("Объявление не найдено")
        continue #Если цена не найдена переходим к следующему товару
    
    # Проврка входим ли в ценовой диапазон
    if not(min_price <= price <= max_price):
        print(f"Цена: {price} в не ценового диапазона: ({min_price} - {max_price})")

    # Название
    title_tag = p_soup.find("h1", class_="ut2-pb__title")
    if title_tag:
        title = title_tag.get_text(strip=True)
        print(f"Название: {title}")
    else:
        print("Объявление не найдено")
    
    # Код товара
    code_tag = p_soup.find("span", id=lambda x: x and x.startswith("product_code_"))
    if code_tag:
        code = code_tag.get_text(strip=True)
        print(f"Код товара: {code}")
    else:
        print("Объявление не найдено")
    
    print(f"id: {item_id} | Цена: {price} | {title}, | Код товара: {code}")
